# Installation (run first)

In [ ]:
install = {'wmt_data' : True,
           'checklist' : True,
           'bert_score' : True,
           'bleurt' : True,
          } 

In [ ]:
if install['wmt_data']:
  !git clone https://github.com/johntzwei/wmt16-19-metrics-shared-task

  # segment-level data
  import pickle
  import pandas as pd

  wmt16 = pickle.load(open('./wmt16-19-metrics-shared-task/data/pickles/wmt16-seg_level-agg.pkl', 'rb'))
  wmt16['year'] = len(wmt16)*[2016]
  wmt17 = pickle.load(open('./wmt16-19-metrics-shared-task/data/pickles/wmt17-seg_level-agg.pkl', 'rb'))
  wmt17['year'] = len(wmt17)*[2017]
  
  wmt_mst_seg = pd.concat([wmt16, wmt17])
  wmt_mst_seg_toen = wmt_mst_seg[wmt_mst_seg.lp.str.endswith('en')]

Cloning into 'wmt16-19-metrics-shared-task'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 107 (delta 15), reused 22 (delta 12), pack-reused 76
Receiving objects: 100% (107/107), 66.96 MiB | 10.99 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [ ]:
if install['checklist']:
  !sudo apt-get install python3-dev default-libmysqlclient-dev
  !pip install mysqlclient
  !pip install checklist

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  default-libmysqlclient-dev libmysqlclient-dev
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 992 kB of archives.
After this operation, 6,023 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmysqlclient-dev amd64 5.7.30-0ubuntu0.18.04.1 [989 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 default-libmysqlclient-dev amd64 1.0.4 [3,736 B]
Fetched 992 kB in 1s (1,202 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd

In [ ]:
if install['bert_score']:
  !pip install bert_score
  from bert_score import score
  score(['Reference'], ['Output'], lang='en')

     |████████████████████████████████| 61kB 1.9MB/s 


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
if install['bleurt']:
  !wget https://storage.googleapis.com/bleurt-oss/bleurt-base-128.zip
  !unzip ./bleurt-base-128.zip
  !git clone https://github.com/google-research/bleurt.git
  !cd bleurt && pip install .

--2020-07-14 20:03:22--  https://storage.googleapis.com/bleurt-oss/bleurt-base-128.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.128, 173.194.217.128, 108.177.13.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405489453 (387M) [application/zip]
Saving to: ‘bleurt-base-128.zip’

bleurt-base-128.zip 100%[===================>] 386.70M   155MB/s    in 2.5s    

2020-07-14 20:03:25 (155 MB/s) - ‘bleurt-base-128.zip’ saved [405489453/405489453]

Archive:  ./bleurt-base-128.zip
   creating: bleurt-base-128/
  inflating: bleurt-base-128/vocab.txt  
  inflating: bleurt-base-128/bert_config.json  
   creating: bleurt-base-128/variables/
  inflating: bleurt-base-128/variables/variables.index  
  inflating: bleurt-base-128/variables/variables.data-00000-of-00001  
  inflating: bleurt-base-128/bleurt_config.json  
  inflating: bleurt-base-128/saved_model.pb

# Scoring WMT data

# Checklist

### Setup

In [ ]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.test_suite import TestSuite
from checklist.expect import Expect
from checklist.pred_wrapper import PredictorWrapper

import numpy as np

# for tensorflow and bleurt
import sys
sys.argv = sys.argv[:1]
from bleurt import score as bleurt_score
bleurt = bleurt_score.BleurtScorer('./bleurt-base-128/')

suite = TestSuite()
editor = Editor()

INFO:tensorflow:Reading checkpoint ./bleurt-base-128/.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Performs basic checks...
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Loading model...
INFO:tensorflow:BLEURT initialized.


In [ ]:
def feed_for_bert_score(data):
  if type(data[0][0]) == tuple:
    flat = sum((list(i) for i in data), [])
    cands, refs = [ i[0] for i in flat ], [ i[1] for i in flat ]
    flat_scores = score(cands, refs, lang='en')[2]
    return [ (i, j) for i,j in zip(flat_scores[::2], flat_scores[1::2]) ]
  if type(data[0][0]) == str:
    cands, refs = [ i[0] for i in data ], [ i[1] for i in data ]
    scores = score(cands, refs, lang='en')[2]
    return scores

def feed_for_bleurt(data):
  if type(data[0][0]) == tuple:
    flat = sum((list(i) for i in data), [])
    cands, refs = [ i[0] for i in flat ], [ i[1] for i in flat ]
    flat_scores = bleurt.score(cands, refs)
    return [ (i, j) for i,j in zip(flat_scores[::2], flat_scores[1::2]) ]
  if type(data[0][0]) == str:
    cands, refs = [ i[0] for i in data ], [ i[1] for i in data ]
    scores = bleurt.score(cands, refs)
    return scores

def wrap_predict(predict_fn):
  def pred_and_conf(inputs):
    preds = predict_fn(inputs)
    confs = np.ones((len(preds), 2))
    return preds, confs
  return pred_and_conf
wrapped_pp = wrap_predict(feed_for_bert_score)

def pairwise_monotonic(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    return pred[0].numpy() - pred[1].numpy()
expect_pairwise_monotonic = Expect.pairwise(pairwise_monotonic)

def high_prediction(x, pred, conf, label=None, meta=None):
    return pred > 1.
expect_high_prediction = Expect.single(high_prediction)

### Top-down

$\text{bleurt}(i, i) > \text{bleurt}(i, \text{bad_perturb_adj}(i))$



In [ ]:
t = editor.template((
    ('{first_name} {last_name} is {mask}.', '{first_name} {last_name} is {mask}.',),
    ('{first_name} {last_name} is {mask}.', '{first_name} {last_name} is {mask2}.',),
    ),
    remove_duplicates=True, 
    nsamples=200)

test = DIR(**t, expect=expect_pairwise_monotonic, name='adjective change', capability='Robustness')
suite.add(test)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


$\text{bleurt}(i,i) > 1$

In [ ]:
t = editor.template(
    ('{first_name} {last_name} is {mask}.', '{first_name} {last_name} is {mask}.',),
    remove_duplicates=True, 
    nsamples=200)
test = DIR(**t, expect=expect_high_prediction, name='same reference and output', capability='Robustness')
suite.add(test)

$\text{bleurt}(i, i) > \text{bleurt}(i, \text{synonym_perturb}(i))$

In [ ]:
t = editor.template((
    ('The {noun} is great.', '{noun} is great.',),
    ('{noun} is great.', '{noun} is {synonym}.',),
    ),
    remove_duplicates=True,
    noun = ['movie', 'TV show', 'anime', 'manga', 'book'],
    synonym = editor.synonyms('This movie is great.', 'great'),
    nsamples=200)

test = DIR(**t, expect=expect_pairwise_monotonic, name='synonym change', capability='Robustness')
suite.add(test)

### Run suite

In [ ]:
suite.run(wrapped_pp, overwrite=True)

Running adjective change
Predicting 198 examples


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

tensor(1.0000) (tensor(1.0000), tensor(0.9494))
tensor(1.0000) (tensor(1.0000), tensor(0.9596))
tensor(1.0000) (tensor(1.0000), tensor(0.9640))
tensor(1.0000) (tensor(1.0000), tensor(0.9290))
tensor(1.0000) (tensor(1.0000), tensor(0.9547))
tensor(1.0000) (tensor(1.0000), tensor(0.9455))
tensor(1.) (tensor(1.), tensor(0.9942))
tensor(1.0000) (tensor(1.0000), tensor(0.9226))
tensor(1.0000) (tensor(1.0000), tensor(0.9287))
tensor(1.0000) (tensor(1.0000), tensor(0.9711))
tensor(1.) (tensor(1.), tensor(0.9492))
tensor(1.) (tensor(1.), tensor(0.9433))
tensor(1.0000) (tensor(1.0000), tensor(0.9764))
tensor(1.0000) (tensor(1.0000), tensor(0.9402))
tensor(1.0000) (tensor(1.0000), tensor(0.9302))
tensor(1.0000) (tensor(1.0000), tensor(0.9901))
tensor(1.0000) (tensor(1.0000), tensor(0.9344))
tensor(1.0000) (tensor(1.0000), tensor(0.9297))
tensor(1.0000) (tensor(1.0000), tensor(0.9431))
tensor(1.0000) (tensor(1.0000), tensor(0.9325))
tensor(1.0000) (tensor(1.0000), tensor(0.9193))
tensor(1.0000) (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Running synonym change
Predicting 200 examples


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

In [ ]:
suite.summary()

Robustness

adjective change
Test cases:      198
Fails (rate):    0 (0.0%)


same reference and output
Test cases:      200
Fails (rate):    0 (0.0%)


synonym change
Test cases:      200
Fails (rate):    0 (0.0%)




